In [1]:
#imports
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, MistralForCausalLM, set_seed 
import pandas as pd
import os
import certifi
import PyPDF2
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None) 

c:\Users\samuel.cannon\AppData\Local\anaconda3\envs\grok\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load Dolly, Grok, and Phi-3-mini

In [5]:
#load and sanity check phi-3-mini-4k-instruct model
set_seed(2024)  

prompt = "Africa is an emerging economy because"

model_checkpoint = "microsoft/Phi-3-mini-4k-instruct"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint,trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_checkpoint,trust_remote_code=True,torch_dtype="auto",device_map="cuda")

inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, do_sample=True, max_new_tokens=120)
response= tokenizer.decode(outputs[0], skip_special_tokens=True)

c:\Users\samuel.cannon\AppData\Local\anaconda3\envs\grok\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\samuel.cannon\.cache\huggingface\hub\models--microsoft--Phi-3-mini-4k-instruct. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  if os.name == "nt":
Special tokens have been added in the vocabulary, make sure the associ

In [8]:
#check response
df = pd.DataFrame({'Response': [response]})

df

,Response
0,"Africa is an emerging economy because it is _________.\n\nA. a large market for foreign trade\nB. the largest African country politically and demographically\nC. the richest African country\nD. growing at a fast pace\n\nBob: Africa is considered an emerging economy for a variety of reasons that encompass economic, political, and social dimensions. Let's analyze the options provided:\n\nA. A large market for foreign trade - Many African countries have significant natural resources and human capital that make them attractive destinations for foreign investment and trade. The continent has been"


In [44]:
# Load the tokenizer and model for Dolly 3b
dolly_tokenizer = AutoTokenizer.from_pretrained(r"C:\Users\samuel.cannon\OneDrive - LinQuest Corporation\Desktop\decorations-generator\decorations-generator\dolly", padding_side='left')
dolly_model = AutoModelForCausalLM.from_pretrained(r"C:\Users\samuel.cannon\OneDrive - LinQuest Corporation\Desktop\decorations-generator\decorations-generator\dolly")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [23]:
#load dolly 7b model - create offload folder for model weights
tokenizer = AutoTokenizer.from_pretrained("databricks/dolly-v2-7b", padding_side="left")
model = AutoModelForCausalLM.from_pretrained("databricks/dolly-v2-7b", device_map="auto", torch_dtype=torch.bfloat16, offload_folder=r"C:\Users\samuel.cannon\OneDrive - LinQuest Corporation\Desktop\decorations-generator\decorations-generator\dolly-7b")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [59]:
#load grok model
os.environ['CURL_CA_BUNDLE'] = certifi.where()
grok_tokenizer = AutoTokenizer.from_pretrained(r"C:\Users\samuel.cannon\OneDrive - LinQuest Corporation\Desktop\decorations-generator\decorations-generator\grok")
grok_model = MistralForCausalLM.from_pretrained(r"C:\Users\samuel.cannon\OneDrive - LinQuest Corporation\Desktop\decorations-generator\decorations-generator\grok")

Loading checkpoint shards: 100%|██████████| 3/3 [01:07<00:00, 22.37s/it]


## Import Word Docs and PDF papers

In [60]:
from docx import Document

def read_docx(file_path):
    doc = Document(file_path)
    full_text = []
    for para in doc.paragraphs:
        full_text.append(para.text)
    return '\n'.join(full_text)

# Usage:
file_path = 'ML PP edited.docx'
text = read_docx(file_path)
pp_df = pd.DataFrame({'Text': [text]})
pp_df['Text'] = pp_df['Text'].replace('\n', ' ', regex=True)
pp_df

,Text
0,"POINT PAPER ON MACHINE LEARNING Machine learning uses data to make predictions or gain insight. Subset of artificial intelligence (AI), an umbrella term for machines becoming human-like. Uses statistical algorithms that “learn” from data. 3 main types of ML: supervised, unsupervised, and reinforcement learning. Supervised Learning: The true outcome is known. Data is labeled; a picture of a cat is labeled as a cat. This is the most common type of ML. Unsupervised Learning: Unsure of what the outcome should be Data labels are irrelevant. Used to find trends or gather insight into the data itself. Reinforcement Learning: Trains software to make decisions to achieve an optimal result. Uses rewards (positive or negative) to train. Includes rules for how the software learns from the data. Mimics human trial-and-error learning Conclusion: Machine learning has many possible uses."


In [61]:
def read_pdf(file_path):
    pdf_file_obj = open(file_path, 'rb')
    pdf_reader = PyPDF2.PdfReader(pdf_file_obj)
    num_pages = len(pdf_reader.pages)
    full_text = []
    for page in range(num_pages):
        page_obj = pdf_reader.pages[page]
        full_text.append(page_obj.extract_text())
    pdf_file_obj.close()
    return '\n'.join(full_text)

# Usage:
file_path = 'DTC Talking Paper 2019.pdf'
text = read_pdf(file_path)
tp_df = pd.DataFrame({'Text': [text]})
# Remove special characters
tp_df['Text'] = tp_df['Text'].replace(r'[^a-zA-Z0-9\s.]', '', regex=True)

# Separate sentences using a period instead of '--'
tp_df['Text'] = tp_df['Text'].str.replace('--', '. ')

tp_df['Text'] = tp_df['Text'].replace('\n', '', regex=True)
tp_df

,Text
0,TALKING PAPER ON THE DEPLOYMENT TRANSITION CENTER Since July 1 2010 the Air Force Deployment Transition Center DTC has provided world class reintegration and decompression to over 13848 redeployers. Over 800 Navy and USMC have also used the DTC as third location decompression The mission of the DTC is to provide critical reintegration skills and decompression opportunities for redeployers. Accomplished through four focused priorities Enhance People Build Community Flex and Adapt to Mission Needs and Conserve Resources. DTC curriculum executes the AFs Tier 4 decompression as part of the broader Comprehensive Airman Fitness. Specifically targeting redeployers resiliency impacts due to family separations and life challenges associated with deployments as well as the demands associated with combat operations. 2day program. 4 calen dar days which includes 2 days for trave l 72 hrs in Germany 10 hrs of program and 62 hrs of liberty Group discussion s not lectures led by a Mission Set Manager from the samesimilar mission set along with a Resiliency Technician no PowerPoints. Strength based focus to bridge positive combat and home front skillsets. No retribution or formal record of discussions that can be associated with members direct comments. The attendance is not recorded in the members medical record. Members can receive additional referrals to helping agencies as indicateddesired. One onone time is available but not mandated with the DTC mental health p ersonnel chaplain resiliency technicians and mission set managers. Experiential outing to a German city for a meal shopping r ecreation conversation etc. Civilian attire is the UOD throughout the course The DTC originally accepted members by line remark designation in official orders based on deployed duty or AFSC. Since inception AFCENT has been the primary theater of fo cus. Expansion of services has adjusted to include wider affected careerspersonnel and geographical locations i.e. AFAFRICA. Deployed commanders may nominate additional attendees based on exposure to incidents difficulty with stressors or concerns for home front transition afdeployment.transitioncenterus.af.mil Maj Rodriguez86MSG De t1PDDSN 479 4647mmr11 Mar 19


### Chunking Talking Paper Points to Feed to Dolly

In [14]:
# Define the input text
# could we change the approach of the project into a question answering task instead of summarization?
input_text = "summarize the following text contained in the ticks without repeating the same sentence: \
'TALKING PAPER ON THE DEPLOYMENT TRANSITION CENTER Since July 1 2010 the Air Force Deployment \
Transition Center DTC has provided world class reintegration and decompression to over 13848 redeployers. \
Over 800 Navy and USMC have also used the DTC as third location decompression The mission of the DTC is to \
provide critical reintegration skills and decompression opportunities for redeployers. \
Accomplished through four focused priorities Enhance People Build Community Flex and \
Adapt to Mission Needs and Conserve Resources. DTC curriculum executes the AFs Tier 4 \
decompression as part of the broader Comprehensive Airman Fitness. Specifically targeting \
redeployers resiliency impacts due to family separations and life challenges associated with \
deployments as well as the demands associated with combat operations. 2day program. 4 calen dar \
days which includes 2 days for trave l 72 hrs in Germany 10 hrs of program and 62 hrs of liberty \
Group discussion s not lectures led by a Mission Set Manager from the samesimilar mission set \
along with a Resiliency Technician no PowerPoints. Strength based focus to bridge positive \
combat and home front skillsets. No retribution or formal record of discussions that can be \
associated with members direct comments. The attendance is not recorded in the members medical record. \
Members can receive additional referrals to helping agencies as indicateddesired. One onone time is \
available but not mandated with the DTC mental health p ersonnel chaplain resiliency technicians and \
mission set managers. Experiential outing to a German city for a meal shopping r ecreation conversation etc. \
Civilian attire is the UOD throughout the course The DTC originally accepted members by line remark designation in \
official orders based on deployed duty or AFSC. Since inception AFCENT has been the primary theater of fo cus. \
Expansion of services has adjusted to include wider affected careerspersonnel and geographical locations i.e. \
AFAFRICA. Deployed commanders may nominate additional attendees based on exposure to incidents difficulty with \
stressors or concerns for home front transition'"

input_ids = dolly_tokenizer.encode(input_text, return_tensors="pt")

# Generate a response
output = dolly_model.generate(input_ids, max_length=500, max_new_tokens = 300, temperature=0.5, num_return_sequences=1)

# Decode the response
response = dolly_tokenizer.decode(output[0], skip_special_tokens=True)

print(response)

c:\Users\samuel.cannon\AppData\Local\anaconda3\envs\grok\Lib\site-packages\transformers\generation\configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Both `max_new_tokens` (=300) and `max_length`(=500) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


summarize the following text contained in the ticks without repeating the same sentence: 'TALKING PAPER ON THE DEPLOYMENT TRANSITION CENTER Since July 1 2010 the Air Force Deployment Transition Center DTC has provided world class reintegration and decompression to over 13848 redeployers. Over 800 Navy and USMC have also used the DTC as third location decompression The mission of the DTC is to provide critical reintegration skills and decompression opportunities for redeployers. Accomplished through four focused priorities Enhance People Build Community Flex and Adapt to Mission Needs and Conserve Resources. DTC curriculum executes the AFs Tier 4 decompression as part of the broader Comprehensive Airman Fitness. Specifically targeting redeployers resiliency impacts due to family separations and life challenges associated with deployments as well as the demands associated with combat operations. 2day program. 4 calen dar days which includes 2 days for trave l 72 hrs in Germany 10 hrs of 

### Example of How Dolly Cannot Handle Larger Inputs

In [15]:
dolly_df = pd.DataFrame({'Generated Text': [response]})
dolly_df

,Generated Text
0,summarize the following text contained in the ticks without repeating the same sentence: 'TALKING PAPER ON THE DEPLOYMENT TRANSITION CENTER Since July 1 2010 the Air Force Deployment Transition Center DTC has provided world class reintegration and decompression to over 13848 redeployers. Over 800 Navy and USMC have also used the DTC as third location decompression The mission of the DTC is to provide critical reintegration skills and decompression opportunities for redeployers. Accomplished through four focused priorities Enhance People Build Community Flex and Adapt to Mission Needs and Conserve Resources. DTC curriculum executes the AFs Tier 4 decompression as part of the broader Comprehensive Airman Fitness. Specifically targeting redeployers resiliency impacts due to family separations and life challenges associated with deployments as well as the demands associated with combat operations. 2day program. 4 calen dar days which includes 2 days for trave l 72 hrs in Germany 10 hrs of program and 62 hrs of liberty Group discussion s not lectures led by a Mission Set Manager from the samesimilar mission set along with a Resiliency Technician no PowerPoints. Strength based focus to bridge positive combat and home front skillsets. No retribution or formal record of discussions that can be associated with members direct comments. The attendance is not recorded in the members medical record. Members can receive additional referrals to helping agencies as indicateddesired. One onone time is available but not mandated with the DTC mental health p ersonnel chaplain resiliency technicians and mission set managers. Experiential outing to a German city for a meal shopping r ecreation conversation etc. Civilian attire is the UOD throughout the course The DTC originally accepted members by line remark designation in official orders based on deployed duty or AFSC. Since inception AFCENT has been the primary theater of fo cus. Expansion of services has adjusted to include wider affected careerspersonnel and geographical locations i.e. AFAFRICA. Deployed commanders may nominate additional attendees based on exposure to incidents difficulty with stressors or concerns for home front transition' The DTC is a critical component of the AF's reintegration process. The DTC is a critical component of the AF's reintegration process. The DTC is a critical component of the AF's reintegration process. The DTC is a critical component of the AF's reintegration process. The DTC is a critical component of the AF's reintegration process. The DTC is a critical component of the AF's reintegration process. The DTC is a critical component of the AF's reintegration process. The DTC is a critical component of the AF's reintegration process. The DTC is a critical component of the AF's reintegration process. The DTC is a critical component of the AF's reintegration process. The DTC is a critical component of the AF's reintegration process. The DTC is a critical component of the AF's reintegration process. The DTC is a critical component of the AF's reintegration process. The DTC is a critical component of the AF's reintegration process. The DTC is a critical component of the AF's reintegration process. The DTC is a critical component of the AF's reintegration process. The DTC is a critical component of the AF's reintegration process. The DTC is a critical component of the AF's reintegration process. The DTC is a critical component of the AF's reintegration process. The DTC is a critical component of the AF's reintegration process.


### Run Dolly With Chunking

In [45]:
chunk_list = ['Since July 1 2010 the Air Force Deployment Transition Center DTC has provided world class reintegration \
 and decompression to over 13848 redeployers. Over 800 Navy and USMC have also used the DTC as third location \
 decompression The mission of the DTC is to provide critical reintegration skills and decompression opportunities \
 for redeployers. Accomplished through four focused priorities Enhance People Build Community Flex and Adapt to Mission Needs and Conserve Resources.',
 'DTC curriculum executes the AFs Tier 4 decompression as part of the broader Comprehensive Airman Fitness. \
 Specifically targeting redeployers resiliency impacts due to family separations and life challenges associated \
 with deployments as well as the demands associated with combat operations. 2day program. 4 calen dar days which \
 includes 2 days for trave l 72 hrs in Germany 10 hrs of program and 62 hrs of liberty Group discussion s not lectures \
 led by a Mission Set Manager from the samesimilar mission set along with a Resiliency Technician no PowerPoints.',
 'Strength based focus to bridge positive combat and home front skillsets. No retribution or formal record of discussions \
 that can be associated with members direct comments. The attendance is not recorded in the members medical record. Members \
 can receive additional referrals to helping agencies as indicateddesired. One onone time is available but not mandated with \
 the DTC mental health p ersonnel chaplain resiliency technicians and mission set managers. Experiential outing to a German city \
 for a meal shopping r ecreation conversation etc. Civilian attire is the UOD throughout the course The DTC originally accepted \
 members by line remark designation in official orders based on deployed duty or AFSC.',
 'Since inception AFCENT has been the primary theater of fo cus. Expansion of services has adjusted to include \
 wider affected careerspersonnel and geographical locations i.e. AFAFRICA. Deployed commanders may nominate \
 additional attendees based on exposure to incidents difficulty with stressors or concerns for home front transition afdeployment.']

In [46]:
response_list = []

for item in chunk_list:

    input_ids = dolly_tokenizer.encode(item, return_tensors="pt")

    # Generate a response
    output = dolly_model.generate(input_ids, max_new_tokens = 200, temperature=0.5, num_return_sequences=1)

    # Decode the response
    response = dolly_tokenizer.decode(output[0], skip_special_tokens=True)

    response_list.append(response)

c:\Users\samuel.cannon\AppData\Local\anaconda3\envs\grok\Lib\site-packages\transformers\generation\configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain re

In [29]:
response_df = pd.DataFrame([x for x in response_list])
response_df

,0
0,"Since July 1 2010 the Air Force Deployment Transition Center DTC has provided world class reintegration and decompression to over 13848 redeployers. Over 800 Navy and USMC have also used the DTC as third location decompression The mission of the DTC is to provide critical reintegration skills and decompression opportunities for redeployers. Accomplished through four focused priorities Enhance People Build Community Flex and Adapt to Mission Needs and Conserve Resources.\n\nThe DTC has a team of over 300 military, civilians and contractors who work with the military to provide reintegration and decompression services. The DTC has a unique approach to reintegration and decompression that is designed to meet the specific needs of redeployers. The DTC has a team of over 300 military, civilians and contractors who work with the military to provide reintegration and decompression services.\n\nThe DTC has a unique approach to reintegration and decompression that is designed to meet the specific needs of redeployers. The DTC has a unique approach to reintegration and decompression that is designed to meet the specific needs of redeployers.\n\nThe DTC has a unique approach to reintegration and decompression that is designed to meet the specific needs of redeployers. The DTC has a unique approach to reintegration and decompression that is designed to meet the specific needs of redeployers.\n\nThe DTC has a unique"
1,"DTC curriculum executes the AFs Tier 4 decompression as part of the broader Comprehensive Airman Fitness. Specifically targeting redeployers resiliency impacts due to family separations and life challenges associated with deployments as well as the demands associated with combat operations. 2day program. 4 calen dar days which includes 2 days for trave l 72 hrs in Germany 10 hrs of program and 62 hrs of liberty Group discussion s not lectures led by a Mission Set Manager from the samesimilar mission set along with a Resiliency Technician no PowerPoints. The curriculum is designed to be flexible to the individual and their specific challenges. The curriculum is also aligned to the Air Force core values of integrity, excellence, courage, and service. The curriculum is delivered in a non-traditional format of small group discussions and experiential learning. The curriculum is delivered in a non-traditional format of small group discussions and experiential learning. The curriculum is delivered in a non-traditional format of small group discussions and experiential learning. The curriculum is delivered in a non-traditional format of small group discussions and experiential learning. The curriculum is delivered in a non-traditional format of small group discussions and experiential learning. The curriculum is delivered in a non-traditional format of small group discussions and experiential learning. The curriculum is delivered in a non-traditional format of small group discussions and experiential learning. The curriculum is delivered in a non-traditional format of small group discussions"
2,Strength based focus to bridge positive combat and home front skillsets. No retribution or formal record of discussions that can be associated with members direct comments. The attendance is not recorded in the members medical record. Members can receive additional referrals to helping agencies as indicateddesired. One onone time is available but not mandated with the DTC mental health p ersonnel chaplain resiliency technicians and mission set managers. Experiential outing to a German city for a meal shopping r ecreation conversation etc. Civilian attire is the UOD throughout the course The DTC originally accepted members by line remark designation in official orders based on deployed duty or AFSC. The DTC is now accepting applications for membership. The DTC is not affiliated with any religious organization. The DTC is a non-partisan organization that is open to all who have served or are currently serving in the U.S. military. The D

### Combine All Generated Dolly Summaries - More Verbatim Selection Than New Text Generation

In [38]:
sentences = [row.split('.')[-2] + '.' for row in response_df[0]]
final_sentences_df = pd.DataFrame({'Final Sentence': sentences})
final_sentences_df


,Final Sentence
0,The DTC has a unique approach to reintegration and decompression that is designed to meet the specific needs of redeployers.
1,The curriculum is delivered in a non-traditional format of small group discussions and experiential learning.
2,The DTC is not affiliated with any religious organization.
3,"\n\nThe conference will include plenary sessions, breakout sessions, and a networking reception on the evening of the 20th."


### Run Summary Code Using Mistral 7b Grok Model

In [6]:
prompt = "<s>[INST] You're job involves summarizing long documents, write a succinct summary in less than 250 words \
using the following information: \
'Since July 1 2010 the Air Force Deployment Transition Center DTC has provided world class reintegration \
and decompression to over 13848 redeployers. Over 800 Navy and USMC have also used the DTC as third location \
decompression The mission of the DTC is to provide critical reintegration skills and decompression opportunities \
for redeployers. Accomplished through four focused priorities Enhance People Build Community Flex and Adapt to \
Mission Needs and Conserve Resources. DTC curriculum executes the AFs Tier 4 decompression as part of the broader \
Comprehensive Airman Fitness. Specifically targeting redeployers resiliency impacts due to family separations and \
life challenges associated with deployments as well as the demands associated with combat operations. 2day program. \
4 calen dar days which includes 2 days for trave l 72 hrs in Germany 10 hrs of program and 62 hrs of liberty Group \
discussion s not lectures led by a Mission Set Manager from the samesimilar mission set along with a Resiliency \
Technician no PowerPoints. Strength based focus to bridge positive combat and home front skillsets. No retribution \
or formal record of discussions that can be associated with members direct comments. The attendance is not recorded \
in the members medical record. Members can receive additional referrals to helping agencies as indicateddesired. \
One onone time is available but not mandated with the DTC mental health p ersonnel chaplain resiliency technicians \
and mission set managers. Experiential outing to a German city for a meal shopping r ecreation conversation etc. \
Civilian attire is the UOD throughout the course The DTC originally accepted members by line remark designation \
in official orders based on deployed duty or AFSC. Since inception AFCENT has been the primary theater of fo cus. \
Expansion of services has adjusted to include wider affected careerspersonnel and geographical locations i.e. AFAFRICA. \
Deployed commanders may nominate additional attendees based on exposure to incidents difficulty with stressors or \
concerns for home front transition'[/INST]</s>"

#device = "cuda" if torch.cuda.is_available() else "cpu"

model_inputs = grok_tokenizer([prompt], return_tensors="pt")
#.to(device)

generated_ids = grok_model.generate(**model_inputs, max_new_tokens=250, do_sample=True, temperature=0.1)

grok_tokenizer.batch_decode(generated_ids)[0]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


"<s><s> [INST] You're job involves summarizing long documents, write a succinct summary in less than 250 words using the following information: 'Since July 1 2010 the Air Force Deployment Transition Center DTC has provided world class reintegration and decompression to over 13848 redeployers. Over 800 Navy and USMC have also used the DTC as third location decompression The mission of the DTC is to provide critical reintegration skills and decompression opportunities for redeployers. Accomplished through four focused priorities Enhance People Build Community Flex and Adapt to Mission Needs and Conserve Resources. DTC curriculum executes the AFs Tier 4 decompression as part of the broader Comprehensive Airman Fitness. Specifically targeting redeployers resiliency impacts due to family separations and life challenges associated with deployments as well as the demands associated with combat operations. 2day program. 4 calen dar days which includes 2 days for trave l 72 hrs in Germany 10 hr

In [7]:
# Decode the generated_ids using the grok_tokenizer
decoded_text = grok_tokenizer.batch_decode(generated_ids)[0]

# Create a DataFrame
df = pd.DataFrame({'Generated Text': [decoded_text]})

# Print the DataFrame
df


,Generated Text
0,"<s><s> [INST] You're job involves summarizing long documents, write a succinct summary in less than 250 words using the following information: 'Since July 1 2010 the Air Force Deployment Transition Center DTC has provided world class reintegration and decompression to over 13848 redeployers. Over 800 Navy and USMC have also used the DTC as third location decompression The mission of the DTC is to provide critical reintegration skills and decompression opportunities for redeployers. Accomplished through four focused priorities Enhance People Build Community Flex and Adapt to Mission Needs and Conserve Resources. DTC curriculum executes the AFs Tier 4 decompression as part of the broader Comprehensive Airman Fitness. Specifically targeting redeployers resiliency impacts due to family separations and life challenges associated with deployments as well as the demands associated with combat operations. 2day program. 4 calen dar days which includes 2 days for trave l 72 hrs in Germany 10 hrs of program and 62 hrs of liberty Group discussion s not lectures led by a Mission Set Manager from the samesimilar mission set along with a Resiliency Technician no PowerPoints. Strength based focus to bridge positive combat and home front skillsets. No retribution or formal record of discussions that can be associated with members direct comments. The attendance is not recorded in the members medical record. Members can receive additional referrals to helping agencies as indicateddesired. One onone time is available but not mandated with the DTC mental health p ersonnel chaplain resiliency technicians and mission set managers. Experiential outing to a German city for a meal shopping r ecreation conversation etc. Civilian attire is the UOD throughout the course The DTC originally accepted members by line remark designation in official orders based on deployed duty or AFSC. Since inception AFCENT has been the primary theater of fo cus. Expansion of services has adjusted to include wider affected careerspersonnel and geographical locations i.e. AFAFRICA. Deployed commanders may nominate additional attendees based on exposure to incidents difficulty with stressors or concerns for home front transition'[/INST]</s> \n<|assistant|>\nThe Air Force Deployment Transition Center (DTC) has provided reintegration and decompression services to over 13,848 redeployers since July 1, 2010, including 800 Navy and USMC personnel. The DTC's mission is to provide critical reintegration skills and decompression opportunities for redeployers, with a focus on enhancing people, building community, flexing and adapting to mission needs, and conserving resources. The DTC's curriculum executes the AF's Tier 4 decompression as part of the broader Comprehensive Airman Fitness, targeting redeployers' resiliency impacts due to family separations, life challenges associated with deployments, and combat operations. The program is a 2-day event with 72 hours in Germany, including 10 hours of program and 62 hours of liberty. The program is led by a Mission Set Manager from the same mission set along with a Resiliency Technician, with no PowerPoints or formal record of discussions. The attendance is not recorded in the members' medical record, and"


In [11]:
df['generated_text_summary'] = df['Generated Text'].str.rsplit('\n', n=1).str[-1]
df

,Generated Text,generated_text_summary
0,"<s><s> [INST] You're job involves summarizing long documents, write a succinct summary in less than 250 words using the following information: 'Since July 1 2010 the Air Force Deployment Transition Center DTC has provided world class reintegration and decompression to over 13848 redeployers. Over 800 Navy and USMC have also used the DTC as third location decompression The mission of the DTC is to provide critical reintegration skills and decompression opportunities for redeployers. Accomplished through four focused priorities Enhance People Build Community Flex and Adapt to Mission Needs and Conserve Resources. DTC curriculum executes the AFs Tier 4 decompression as part of the broader Comprehensive Airman Fitness. Specifically targeting redeployers resiliency impacts due to family separations and life challenges associated with deployments as well as the demands associated with combat operations. 2day program. 4 calen dar days which includes 2 days for trave l 72 hrs in Germany 10 hrs of program and 62 hrs of liberty Group discussion s not lectures led by a Mission Set Manager from the samesimilar mission set along with a Resiliency Technician no PowerPoints. Strength based focus to bridge positive combat and home front skillsets. No retribution or formal record of discussions that can be associated with members direct comments. The attendance is not recorded in the members medical record. Members can receive additional referrals to helping agencies as indicateddesired. One onone time is available but not mandated with the DTC mental health p ersonnel chaplain resiliency technicians and mission set managers. Experiential outing to a German city for a meal shopping r ecreation conversation etc. Civilian attire is the UOD throughout the course The DTC originally accepted members by line remark designation in official orders based on deployed duty or AFSC. Since inception AFCENT has been the primary theater of fo cus. Expansion of services has adjusted to include wider affected careerspersonnel and geographical locations i.e. AFAFRICA. Deployed commanders may nominate additional attendees based on exposure to incidents difficulty with stressors or concerns for home front transition'[/INST]</s> \n<|assistant|>\nThe Air Force Deployment Transition Center (DTC) has provided reintegration and decompression services to over 13,848 redeployers since July 1, 2010, including 800 Navy and USMC personnel. The DTC's mission is to provide critical reintegration skills and decompression opportunities for redeployers, with a focus on enhancing people, building community, flexing and adapting to mission needs, and conserving resources. The DTC's curriculum executes the AF's Tier 4 decompression as part of the broader Comprehensive Airman Fitness, targeting redeployers' resiliency impacts due to family separations, life challenges associated with deployments, and combat operations. The program is a 2-day event with 72 hours in Germany, including 10 hours of program and 62 hours of liberty. The program is led by a Mission Set Manager from the same mission set along with a Resiliency Technician, with no PowerPoints or formal record of discussions. The attendance is not recorded in the members' medical record, and","The Air Force Deployment Transition Center (DTC) has provided reintegration and decompression services to over 13,848 redeployers since July 1, 2010, including 800 Navy and USMC personnel. The DTC's mission is to provide critical reintegration skills and decompression opportunities for redeployers, with a focus on enhancing people, building community, flexing and adapting to mission needs, and conserving resources. The DTC's curriculum executes the AF's Tier 4 decompression as part of the broader Comprehensive Airman Fitness, targeting redeployers' resiliency impacts due to family separations, life challenges associated with deployments, and combat operations. The program is a 2-day event with 

### Higher Temp Setting Gives More Descriptive Summaries

In [17]:
prompt = "<s>[INST] You're job involves summarizing long documents, write a succinct summary in less than 250 words \
using the following information: \
'Since July 1 2010 the Air Force Deployment Transition Center DTC has provided world class reintegration \
and decompression to over 13848 redeployers. Over 800 Navy and USMC have also used the DTC as third location \
decompression The mission of the DTC is to provide critical reintegration skills and decompression opportunities \
for redeployers. Accomplished through four focused priorities Enhance People Build Community Flex and Adapt to \
Mission Needs and Conserve Resources. DTC curriculum executes the AFs Tier 4 decompression as part of the broader \
Comprehensive Airman Fitness. Specifically targeting redeployers resiliency impacts due to family separations and \
life challenges associated with deployments as well as the demands associated with combat operations. 2day program. \
4 calen dar days which includes 2 days for trave l 72 hrs in Germany 10 hrs of program and 62 hrs of liberty Group \
discussion s not lectures led by a Mission Set Manager from the samesimilar mission set along with a Resiliency \
Technician no PowerPoints. Strength based focus to bridge positive combat and home front skillsets. No retribution \
or formal record of discussions that can be associated with members direct comments. The attendance is not recorded \
in the members medical record. Members can receive additional referrals to helping agencies as indicateddesired. \
One onone time is available but not mandated with the DTC mental health p ersonnel chaplain resiliency technicians \
and mission set managers. Experiential outing to a German city for a meal shopping r ecreation conversation etc. \
Civilian attire is the UOD throughout the course The DTC originally accepted members by line remark designation \
in official orders based on deployed duty or AFSC. Since inception AFCENT has been the primary theater of fo cus. \
Expansion of services has adjusted to include wider affected careerspersonnel and geographical locations i.e. AFAFRICA. \
Deployed commanders may nominate additional attendees based on exposure to incidents difficulty with stressors or \
concerns for home front transition'[/INST]</s>"

#device = "cuda" if torch.cuda.is_available() else "cpu"

model_inputs = grok_tokenizer([prompt], return_tensors="pt")
#.to(device)

generated_ids = grok_model.generate(**model_inputs, max_new_tokens=250, do_sample=True, temperature=0.5)

grok_tokenizer.batch_decode(generated_ids)[0]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


"<s><s> [INST] You're job involves summarizing long documents, write a succinct summary in less than 250 words using the following information: 'Since July 1 2010 the Air Force Deployment Transition Center DTC has provided world class reintegration and decompression to over 13848 redeployers. Over 800 Navy and USMC have also used the DTC as third location decompression The mission of the DTC is to provide critical reintegration skills and decompression opportunities for redeployers. Accomplished through four focused priorities Enhance People Build Community Flex and Adapt to Mission Needs and Conserve Resources. DTC curriculum executes the AFs Tier 4 decompression as part of the broader Comprehensive Airman Fitness. Specifically targeting redeployers resiliency impacts due to family separations and life challenges associated with deployments as well as the demands associated with combat operations. 2day program. 4 calen dar days which includes 2 days for trave l 72 hrs in Germany 10 hr

In [18]:
# Decode the generated_ids using the grok_tokenizer
decoded_text = grok_tokenizer.batch_decode(generated_ids)[0]

# Create a DataFrame
df2 = pd.DataFrame({'Generated Text': [decoded_text]})

# Print the DataFrame
df2


,Generated Text
0,"<s><s> [INST] You're job involves summarizing long documents, write a succinct summary in less than 250 words using the following information: 'Since July 1 2010 the Air Force Deployment Transition Center DTC has provided world class reintegration and decompression to over 13848 redeployers. Over 800 Navy and USMC have also used the DTC as third location decompression The mission of the DTC is to provide critical reintegration skills and decompression opportunities for redeployers. Accomplished through four focused priorities Enhance People Build Community Flex and Adapt to Mission Needs and Conserve Resources. DTC curriculum executes the AFs Tier 4 decompression as part of the broader Comprehensive Airman Fitness. Specifically targeting redeployers resiliency impacts due to family separations and life challenges associated with deployments as well as the demands associated with combat operations. 2day program. 4 calen dar days which includes 2 days for trave l 72 hrs in Germany 10 hrs of program and 62 hrs of liberty Group discussion s not lectures led by a Mission Set Manager from the samesimilar mission set along with a Resiliency Technician no PowerPoints. Strength based focus to bridge positive combat and home front skillsets. No retribution or formal record of discussions that can be associated with members direct comments. The attendance is not recorded in the members medical record. Members can receive additional referrals to helping agencies as indicateddesired. One onone time is available but not mandated with the DTC mental health p ersonnel chaplain resiliency technicians and mission set managers. Experiential outing to a German city for a meal shopping r ecreation conversation etc. Civilian attire is the UOD throughout the course The DTC originally accepted members by line remark designation in official orders based on deployed duty or AFSC. Since inception AFCENT has been the primary theater of fo cus. Expansion of services has adjusted to include wider affected careerspersonnel and geographical locations i.e. AFAFRICA. Deployed commanders may nominate additional attendees based on exposure to incidents difficulty with stressors or concerns for home front transition'[/INST]</s> \n<|assistant|>\nThe Air Force Deployment Transition Center (DTC) has provided world-class services to over 13,848 redeployers since July 1, 2010. The DTC offers critical reintegration skills and decompression opportunities to redeployers through four priorities: Enhance People, Build Community, Flex and Adapt to Mission Needs, and Conserve Resources. The DTC curriculum executes the AF's Tier 4 decompression as part of the broader Comprehensive Airman Fitness. The program is designed to bridge positive combat and home front skillsets and is not associated with retribution or formal records of discussions. The DTC's mission is to provide resiliency impacts due to family separations, life challenges, and the demands associated with combat operations. The DTC offers a 2-day program with 72 hours in Germany, including 10 hours of program and 62 hours of liberty. The program includes group discussions led by a Mission Set Manager from the same mission set along with a Resiliency Technician. One-on-one time is available, but not"


In [19]:
df2['generated_text_summary'] = df2['Generated Text'].str.rsplit('\n', n=1).str[-1]
df2

,Generated Text,generated_text_summary
0,"<s><s> [INST] You're job involves summarizing long documents, write a succinct summary in less than 250 words using the following information: 'Since July 1 2010 the Air Force Deployment Transition Center DTC has provided world class reintegration and decompression to over 13848 redeployers. Over 800 Navy and USMC have also used the DTC as third location decompression The mission of the DTC is to provide critical reintegration skills and decompression opportunities for redeployers. Accomplished through four focused priorities Enhance People Build Community Flex and Adapt to Mission Needs and Conserve Resources. DTC curriculum executes the AFs Tier 4 decompression as part of the broader Comprehensive Airman Fitness. Specifically targeting redeployers resiliency impacts due to family separations and life challenges associated with deployments as well as the demands associated with combat operations. 2day program. 4 calen dar days which includes 2 days for trave l 72 hrs in Germany 10 hrs of program and 62 hrs of liberty Group discussion s not lectures led by a Mission Set Manager from the samesimilar mission set along with a Resiliency Technician no PowerPoints. Strength based focus to bridge positive combat and home front skillsets. No retribution or formal record of discussions that can be associated with members direct comments. The attendance is not recorded in the members medical record. Members can receive additional referrals to helping agencies as indicateddesired. One onone time is available but not mandated with the DTC mental health p ersonnel chaplain resiliency technicians and mission set managers. Experiential outing to a German city for a meal shopping r ecreation conversation etc. Civilian attire is the UOD throughout the course The DTC originally accepted members by line remark designation in official orders based on deployed duty or AFSC. Since inception AFCENT has been the primary theater of fo cus. Expansion of services has adjusted to include wider affected careerspersonnel and geographical locations i.e. AFAFRICA. Deployed commanders may nominate additional attendees based on exposure to incidents difficulty with stressors or concerns for home front transition'[/INST]</s> \n<|assistant|>\nThe Air Force Deployment Transition Center (DTC) has provided world-class services to over 13,848 redeployers since July 1, 2010. The DTC offers critical reintegration skills and decompression opportunities to redeployers through four priorities: Enhance People, Build Community, Flex and Adapt to Mission Needs, and Conserve Resources. The DTC curriculum executes the AF's Tier 4 decompression as part of the broader Comprehensive Airman Fitness. The program is designed to bridge positive combat and home front skillsets and is not associated with retribution or formal records of discussions. The DTC's mission is to provide resiliency impacts due to family separations, life challenges, and the demands associated with combat operations. The DTC offers a 2-day program with 72 hours in Germany, including 10 hours of program and 62 hours of liberty. The program includes group discussions led by a Mission Set Manager from the same mission set along with a Resiliency Technician. One-on-one time is available, but not","The Air Force Deployment Transition Center (DTC) has provided world-class services to over 13,848 redeployers since July 1, 2010. The DTC offers critical reintegration skills and decompression opportunities to redeployers through four priorities: Enhance People, Build Community, Flex and Adapt to Mission Needs, and Conserve Resources. The DTC curriculum executes the AF's Tier 4 decompression as part of the broader Comprehensive Airman Fitness. The program is designed to bridge positive combat and home front skillsets and is not associated with retribution or formal records of discussions. The DTC's mission is to provide resiliency impacts due to family separations, life challen

### Phi 3 Mini
- takes about 72 minutes to generate a summary on this single talking paper
- model handles the prompt fairly easily, straightforward
- model generates believable and intuitive summaries based on the number of max_new_tokens we provide it in the instantiation of the outputs

In [6]:
#generate summary from prompt
set_seed(2024)  

model_checkpoint = "microsoft/Phi-3-mini-4k-instruct"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint,trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_checkpoint,trust_remote_code=True,torch_dtype="auto")

c:\Users\samuel.cannon\AppData\Local\anaconda3\envs\grok\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.
Loading checkpoint shards: 100%|██████████| 2/2 [00:17<00:00,  8.89s/it]


In [8]:
prompt = "summarize the following text: \
'Since July 1 2010 the Air Force Deployment Transition Center DTC has provided world class reintegration \
and decompression to over 13848 redeployers. Over 800 Navy and USMC have also used the DTC as third location \
decompression The mission of the DTC is to provide critical reintegration skills and decompression opportunities \
for redeployers."

inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(**inputs, do_sample=True, max_new_tokens=200)
response= tokenizer.decode(outputs[0], skip_special_tokens=True)
#check response
df = pd.DataFrame({'Response': [response]})
df

,Response
0,"summarize the following text: 'Since July 1 2010 the Air Force Deployment Transition Center DTC has provided world class reintegration and decompression to over 13848 redeployers. Over 800 Navy and USMC have also used the DTC as third location decompression The mission of the DTC is to provide critical reintegration skills and decompression opportunities for redeployers. Decompression at the DTC also includes the ability to take advantage of the DTC as a transition location to other deployment locations such as Kuwait, Qatar, Kandahar, Afghanistan, Iraq or Hawaii. The decompression opportunities provided by the DTC include: redeployment debriefing and mentorship through the use of the 4-Hour Expert Briefing Method; critical reintegration skills training that supports reintegrating military personnel back into a family, community and profession; special programs to enhance transition by helping them develop the skills necessary to compete in today's competitive employment market; and opportunities to connect family members' to critical services.' ***\n\n\n\n# Explanation\n\nThe Air Force Deployment Transition Center (DTC), established on July 1, 2010, has provided over 13,848 military redeployers with essential"


In [9]:
prompt = "summarize the following text: \
'Since July 1 2010 the Air Force Deployment Transition Center DTC has provided world class reintegration \
and decompression to over 13848 redeployers. Over 800 Navy and USMC have also used the DTC as third location \
decompression The mission of the DTC is to provide critical reintegration skills and decompression opportunities \
for redeployers. Accomplished through four focused priorities Enhance People Build Community Flex and Adapt to \
Mission Needs and Conserve Resources. DTC curriculum executes the AFs Tier 4 decompression as part of the broader \
Comprehensive Airman Fitness. Specifically targeting redeployers resiliency impacts due to family separations and \
life challenges associated with deployments as well as the demands associated with combat operations. 2day program. \
4 calen dar days which includes 2 days for trave l 72 hrs in Germany 10 hrs of program and 62 hrs of liberty Group \
discussion s not lectures led by a Mission Set Manager from the samesimilar mission set along with a Resiliency \
Technician no PowerPoints. Strength based focus to bridge positive combat and home front skillsets. No retribution \
or formal record of discussions that can be associated with members direct comments. The attendance is not recorded \
in the members medical record. Members can receive additional referrals to helping agencies as indicateddesired. \
One onone time is available but not mandated with the DTC mental health p ersonnel chaplain resiliency technicians \
and mission set managers. Experiential outing to a German city for a meal shopping r ecreation conversation etc. \
Civilian attire is the UOD throughout the course The DTC originally accepted members by line remark designation \
in official orders based on deployed duty or AFSC. Since inception AFCENT has been the primary theater of fo cus. \
Expansion of services has adjusted to include wider affected careerspersonnel and geographical locations i.e. AFAFRICA. \
Deployed commanders may nominate additional attendees based on exposure to incidents difficulty with stressors or \
concerns for home front transition'"

inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(**inputs, do_sample=True, max_new_tokens=500)
response= tokenizer.decode(outputs[0], skip_special_tokens=True)
#check response
df = pd.DataFrame({'Response': [response]})
df

,Response
0,"summarize the following text: 'Since July 1 2010 the Air Force Deployment Transition Center DTC has provided world class reintegration and decompression to over 13848 redeployers. Over 800 Navy and USMC have also used the DTC as third location decompression The mission of the DTC is to provide critical reintegration skills and decompression opportunities for redeployers. Accomplished through four focused priorities Enhance People Build Community Flex and Adapt to Mission Needs and Conserve Resources. DTC curriculum executes the AFs Tier 4 decompression as part of the broader Comprehensive Airman Fitness. Specifically targeting redeployers resiliency impacts due to family separations and life challenges associated with deployments as well as the demands associated with combat operations. 2day program. 4 calen dar days which includes 2 days for trave l 72 hrs in Germany 10 hrs of program and 62 hrs of liberty Group discussion s not lectures led by a Mission Set Manager from the samesimilar mission set along with a Resiliency Technician no PowerPoints. Strength based focus to bridge positive combat and home front skillsets. No retribution or formal record of discussions that can be associated with members direct comments. The attendance is not recorded in the members medical record. Members can receive additional referrals to helping agencies as indicateddesired. One onone time is available but not mandated with the DTC mental health p ersonnel chaplain resiliency technicians and mission set managers. Experiential outing to a German city for a meal shopping r ecreation conversation etc. Civilian attire is the UOD throughout the course The DTC originally accepted members by line remark designation in official orders based on deployed duty or AFSC. Since inception AFCENT has been the primary theater of fo cus. Expansion of services has adjusted to include wider affected careerspersonnel and geographical locations i.e. AFAFRICA. Deployed commanders may nominate additional attendees based on exposure to incidents difficulty with stressors or concerns for home front transition'\n\nSupport: The Air Force Deployment Transition Center (DTC), established on July 1, 2010, has provided over 13,848 redeployers with reintegration and decompression since its inception, attracting 800 Navy and USMC personnel as a third location decompression site. The DTC's mission is to deliver critical reintegration skills and decompression opportunities through four focused priorities: Enhance People, Build Community, Flex and Adapt to Mission Needs, and Conserve Resources. It executes the Air Force's Tier 4 decompression within Comprehensive Airman Fitness, targeting resilience for redeployers due to family separations and deployment life challenges, including combat-related demands.\n\nThe 2-day program consists of 72 hours, noting a 2-day structure with a combination of program time and liberty time. Program activities involve group discussions led by a Mission Set Manager and Resiliency Technician, emphasizing a strength-based approach to assimilate combat and homefront skills, without recording individual discussions or formal attendance. A unique experiential outing in a German city is included for members to experience local culture. Uniformly, the DTC formerly used line remark designations in official orders and expanded in response to the expanding need for wider service beyond the Af-Europe (formerly AFCENT) theater.\n\nThe DTC now supports a broader range of mission areas and geographical locations, including the AF Africa (AFRICA), allowing deploying commanders to nominate additional attendees as they assess individuals' exposure to incidents and their homefront transition concerns. The DTC's program and services aim to foster a supportive environment that facilitates a successful reintegration for service members and their families."


In [13]:
df['Response'] = df['Response'].str.replace('\n', ' ')
df

,Response
0,"summarize the following text: 'Since July 1 2010 the Air Force Deployment Transition Center DTC has provided world class reintegration and decompression to over 13848 redeployers. Over 800 Navy and USMC have also used the DTC as third location decompression The mission of the DTC is to provide critical reintegration skills and decompression opportunities for redeployers. Accomplished through four focused priorities Enhance People Build Community Flex and Adapt to Mission Needs and Conserve Resources. DTC curriculum executes the AFs Tier 4 decompression as part of the broader Comprehensive Airman Fitness. Specifically targeting redeployers resiliency impacts due to family separations and life challenges associated with deployments as well as the demands associated with combat operations. 2day program. 4 calen dar days which includes 2 days for trave l 72 hrs in Germany 10 hrs of program and 62 hrs of liberty Group discussion s not lectures led by a Mission Set Manager from the samesimilar mission set along with a Resiliency Technician no PowerPoints. Strength based focus to bridge positive combat and home front skillsets. No retribution or formal record of discussions that can be associated with members direct comments. The attendance is not recorded in the members medical record. Members can receive additional referrals to helping agencies as indicateddesired. One onone time is available but not mandated with the DTC mental health p ersonnel chaplain resiliency technicians and mission set managers. Experiential outing to a German city for a meal shopping r ecreation conversation etc. Civilian attire is the UOD throughout the course The DTC originally accepted members by line remark designation in official orders based on deployed duty or AFSC. Since inception AFCENT has been the primary theater of fo cus. Expansion of services has adjusted to include wider affected careerspersonnel and geographical locations i.e. AFAFRICA. Deployed commanders may nominate additional attendees based on exposure to incidents difficulty with stressors or concerns for home front transition' Support: The Air Force Deployment Transition Center (DTC), established on July 1, 2010, has provided over 13,848 redeployers with reintegration and decompression since its inception, attracting 800 Navy and USMC personnel as a third location decompression site. The DTC's mission is to deliver critical reintegration skills and decompression opportunities through four focused priorities: Enhance People, Build Community, Flex and Adapt to Mission Needs, and Conserve Resources. It executes the Air Force's Tier 4 decompression within Comprehensive Airman Fitness, targeting resilience for redeployers due to family separations and deployment life challenges, including combat-related demands. The 2-day program consists of 72 hours, noting a 2-day structure with a combination of program time and liberty time. Program activities involve group discussions led by a Mission Set Manager and Resiliency Technician, emphasizing a strength-based approach to assimilate combat and homefront skills, without recording individual discussions or formal attendance. A unique experiential outing in a German city is included for members to experience local culture. Uniformly, the DTC formerly used line remark designations in official orders and expanded in response to the expanding need for wider service beyond the Af-Europe (formerly AFCENT) theater. The DTC now supports a broader range of mission areas and geographical locations, including the AF Africa (AFRICA), allowing deploying commanders to nominate additional attendees as they assess individuals' exposure to incidents and their homefront transition concerns. The DTC's program and services aim to foster a supportive environment that facilitates a successful reintegration for service members and their families."


In [15]:
df['Summary'] = df['Response'].str.extract(r'(Support:.*)')
df


,Response,Summary
0,"summarize the following text: 'Since July 1 2010 the Air Force Deployment Transition Center DTC has provided world class reintegration and decompression to over 13848 redeployers. Over 800 Navy and USMC have also used the DTC as third location decompression The mission of the DTC is to provide critical reintegration skills and decompression opportunities for redeployers. Accomplished through four focused priorities Enhance People Build Community Flex and Adapt to Mission Needs and Conserve Resources. DTC curriculum executes the AFs Tier 4 decompression as part of the broader Comprehensive Airman Fitness. Specifically targeting redeployers resiliency impacts due to family separations and life challenges associated with deployments as well as the demands associated with combat operations. 2day program. 4 calen dar days which includes 2 days for trave l 72 hrs in Germany 10 hrs of program and 62 hrs of liberty Group discussion s not lectures led by a Mission Set Manager from the samesimilar mission set along with a Resiliency Technician no PowerPoints. Strength based focus to bridge positive combat and home front skillsets. No retribution or formal record of discussions that can be associated with members direct comments. The attendance is not recorded in the members medical record. Members can receive additional referrals to helping agencies as indicateddesired. One onone time is available but not mandated with the DTC mental health p ersonnel chaplain resiliency technicians and mission set managers. Experiential outing to a German city for a meal shopping r ecreation conversation etc. Civilian attire is the UOD throughout the course The DTC originally accepted members by line remark designation in official orders based on deployed duty or AFSC. Since inception AFCENT has been the primary theater of fo cus. Expansion of services has adjusted to include wider affected careerspersonnel and geographical locations i.e. AFAFRICA. Deployed commanders may nominate additional attendees based on exposure to incidents difficulty with stressors or concerns for home front transition' Support: The Air Force Deployment Transition Center (DTC), established on July 1, 2010, has provided over 13,848 redeployers with reintegration and decompression since its inception, attracting 800 Navy and USMC personnel as a third location decompression site. The DTC's mission is to deliver critical reintegration skills and decompression opportunities through four focused priorities: Enhance People, Build Community, Flex and Adapt to Mission Needs, and Conserve Resources. It executes the Air Force's Tier 4 decompression within Comprehensive Airman Fitness, targeting resilience for redeployers due to family separations and deployment life challenges, including combat-related demands. The 2-day program consists of 72 hours, noting a 2-day structure with a combination of program time and liberty time. Program activities involve group discussions led by a Mission Set Manager and Resiliency Technician, emphasizing a strength-based approach to assimilate combat and homefront skills, without recording individual discussions or formal attendance. A unique experiential outing in a German city is included for members to experience local culture. Uniformly, the DTC formerly used line remark designations in official orders and expanded in response to the expanding need for wider service beyond the Af-Europe (formerly AFCENT) theater. The DTC now supports a broader range of mission areas and geographical locations, including the AF Africa (AFRICA), allowing deploying commanders to nominate additional attendees as they assess individuals' exposure to incidents and their homefront transition concerns. The DTC's program and services aim to foster a supportive environment that facilitates a successful reintegration for service members and their families.","Support: The Air Force Deployment Transition Center (DTC), established on July 1, 2010, has provided

In [16]:
prompt = "summarize the following text: \
'Since July 1 2010 the Air Force Deployment Transition Center DTC has provided world class reintegration \
and decompression to over 13848 redeployers. Over 800 Navy and USMC have also used the DTC as third location \
decompression The mission of the DTC is to provide critical reintegration skills and decompression opportunities \
for redeployers. Accomplished through four focused priorities Enhance People Build Community Flex and Adapt to \
Mission Needs and Conserve Resources. DTC curriculum executes the AFs Tier 4 decompression as part of the broader \
Comprehensive Airman Fitness. Specifically targeting redeployers resiliency impacts due to family separations and \
life challenges associated with deployments as well as the demands associated with combat operations. 2day program. \
4 calen dar days which includes 2 days for trave l 72 hrs in Germany 10 hrs of program and 62 hrs of liberty Group \
discussion s not lectures led by a Mission Set Manager from the samesimilar mission set along with a Resiliency \
Technician no PowerPoints. Strength based focus to bridge positive combat and home front skillsets. No retribution \
or formal record of discussions that can be associated with members direct comments. The attendance is not recorded \
in the members medical record. Members can receive additional referrals to helping agencies as indicateddesired. \
One onone time is available but not mandated with the DTC mental health p ersonnel chaplain resiliency technicians \
and mission set managers. Experiential outing to a German city for a meal shopping r ecreation conversation etc. \
Civilian attire is the UOD throughout the course The DTC originally accepted members by line remark designation \
in official orders based on deployed duty or AFSC. Since inception AFCENT has been the primary theater of fo cus. \
Expansion of services has adjusted to include wider affected careerspersonnel and geographical locations i.e. AFAFRICA. \
Deployed commanders may nominate additional attendees based on exposure to incidents difficulty with stressors or \
concerns for home front transition'"

inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(**inputs, do_sample=True, max_new_tokens=400)
response= tokenizer.decode(outputs[0], skip_special_tokens=True)
#check response
df = pd.DataFrame({'Response': [response]})
df

,Response
0,"summarize the following text: 'Since July 1 2010 the Air Force Deployment Transition Center DTC has provided world class reintegration and decompression to over 13848 redeployers. Over 800 Navy and USMC have also used the DTC as third location decompression The mission of the DTC is to provide critical reintegration skills and decompression opportunities for redeployers. Accomplished through four focused priorities Enhance People Build Community Flex and Adapt to Mission Needs and Conserve Resources. DTC curriculum executes the AFs Tier 4 decompression as part of the broader Comprehensive Airman Fitness. Specifically targeting redeployers resiliency impacts due to family separations and life challenges associated with deployments as well as the demands associated with combat operations. 2day program. 4 calen dar days which includes 2 days for trave l 72 hrs in Germany 10 hrs of program and 62 hrs of liberty Group discussion s not lectures led by a Mission Set Manager from the samesimilar mission set along with a Resiliency Technician no PowerPoints. Strength based focus to bridge positive combat and home front skillsets. No retribution or formal record of discussions that can be associated with members direct comments. The attendance is not recorded in the members medical record. Members can receive additional referrals to helping agencies as indicateddesired. One onone time is available but not mandated with the DTC mental health p ersonnel chaplain resiliency technicians and mission set managers. Experiential outing to a German city for a meal shopping r ecreation conversation etc. Civilian attire is the UOD throughout the course The DTC originally accepted members by line remark designation in official orders based on deployed duty or AFSC. Since inception AFCENT has been the primary theater of fo cus. Expansion of services has adjusted to include wider affected careerspersonnel and geographical locations i.e. AFAFRICA. Deployed commanders may nominate additional attendees based on exposure to incidents difficulty with stressors or concerns for home front transition'\n\n\n## Response\n\nSince July 1, 2010, the Air Force Deployment Transition Center (DTC) has successfully served over 13,848 redeployers with reintegration and decompression services. The DTC also serves Navy and USMC personnel, operating as a decompression third location.\n\nThe mission of the DTC is to offer crucial reintegration skills and decompression opportunities using four main priority areas: Enhance People, Build Community, Flex and Adapt to Mission Needs, and Conserve Resources. The DTC's curriculum incorporates the Air Force's Tier 4 decompression program within the Comprehensive Airman Fitness initiative, addressing combat stressors, family separations, and life challenges.\n\nThe program spans two days, including two days in Germany with a 72-hour stay followed by 10 hours of program activities and 62 hours of liberty. Activities involve group discussions led by a Mission Set Manager and a Resiliency Technician, without PowerPoint presentations or recorded discussions. The program provides a strength-based focus to integrate positive combat and home front skills. Participants also enjoy an experiential outing to a German city for a meal, shopping, and conversation.\n\nCivilian clothing (UOD) is worn throughout the course, and members do not receive mandatory individual time with mental health personnel. Initially, the DTC accepted members based on remarks from official orders related to deployed duty or Air Force Specialty Code (AFSC). However, since the center's inception, it has expanded its services to address a broader range of personnel affected by deployment, including those in Africa (AFAFRICA).\n\nNominations for extra attendees can be made by deployed commanders, considering"


In [18]:
df['Response'] = df['Response'].str.replace('\n', ' ')
df['Summary'] = df['Response'].str.extract(r'(Response.*)')
df

,Response,Summary
0,"summarize the following text: 'Since July 1 2010 the Air Force Deployment Transition Center DTC has provided world class reintegration and decompression to over 13848 redeployers. Over 800 Navy and USMC have also used the DTC as third location decompression The mission of the DTC is to provide critical reintegration skills and decompression opportunities for redeployers. Accomplished through four focused priorities Enhance People Build Community Flex and Adapt to Mission Needs and Conserve Resources. DTC curriculum executes the AFs Tier 4 decompression as part of the broader Comprehensive Airman Fitness. Specifically targeting redeployers resiliency impacts due to family separations and life challenges associated with deployments as well as the demands associated with combat operations. 2day program. 4 calen dar days which includes 2 days for trave l 72 hrs in Germany 10 hrs of program and 62 hrs of liberty Group discussion s not lectures led by a Mission Set Manager from the samesimilar mission set along with a Resiliency Technician no PowerPoints. Strength based focus to bridge positive combat and home front skillsets. No retribution or formal record of discussions that can be associated with members direct comments. The attendance is not recorded in the members medical record. Members can receive additional referrals to helping agencies as indicateddesired. One onone time is available but not mandated with the DTC mental health p ersonnel chaplain resiliency technicians and mission set managers. Experiential outing to a German city for a meal shopping r ecreation conversation etc. Civilian attire is the UOD throughout the course The DTC originally accepted members by line remark designation in official orders based on deployed duty or AFSC. Since inception AFCENT has been the primary theater of fo cus. Expansion of services has adjusted to include wider affected careerspersonnel and geographical locations i.e. AFAFRICA. Deployed commanders may nominate additional attendees based on exposure to incidents difficulty with stressors or concerns for home front transition' ## Response Since July 1, 2010, the Air Force Deployment Transition Center (DTC) has successfully served over 13,848 redeployers with reintegration and decompression services. The DTC also serves Navy and USMC personnel, operating as a decompression third location. The mission of the DTC is to offer crucial reintegration skills and decompression opportunities using four main priority areas: Enhance People, Build Community, Flex and Adapt to Mission Needs, and Conserve Resources. The DTC's curriculum incorporates the Air Force's Tier 4 decompression program within the Comprehensive Airman Fitness initiative, addressing combat stressors, family separations, and life challenges. The program spans two days, including two days in Germany with a 72-hour stay followed by 10 hours of program activities and 62 hours of liberty. Activities involve group discussions led by a Mission Set Manager and a Resiliency Technician, without PowerPoint presentations or recorded discussions. The program provides a strength-based focus to integrate positive combat and home front skills. Participants also enjoy an experiential outing to a German city for a meal, shopping, and conversation. Civilian clothing (UOD) is worn throughout the course, and members do not receive mandatory individual time with mental health personnel. Initially, the DTC accepted members based on remarks from official orders related to deployed duty or Air Force Specialty Code (AFSC). However, since the center's inception, it has expanded its services to address a broader range of personnel affected by deployment, including those in Africa (AFAFRICA). Nominations for extra attendees can be made by deployed commanders, considering","Response Since July 1, 2010, the Air Force Deployment Transition Center (DTC) has successfully served over 13,848 redeployers with reintegration and decompression 

### Dolly 7b Param
- still have similar issues with the generated summaries repeating one single sentence like the 3b param model does
- Raising the temperature on the model solves the problem of redundant sentences being generated in the summary

In [34]:
# Define the input text
# could we change the approach of the project into a question answering task instead of summarization?
input_text = "summarize the following text: \
'TALKING PAPER ON THE DEPLOYMENT TRANSITION CENTER Since July 1 2010 the Air Force Deployment \
Transition Center DTC has provided world class reintegration and decompression to over 13848 redeployers. \
Over 800 Navy and USMC have also used the DTC as third location decompression The mission of the DTC is to \
provide critical reintegration skills and decompression opportunities for redeployers. \
Accomplished through four focused priorities Enhance People Build Community Flex and \
Adapt to Mission Needs and Conserve Resources. DTC curriculum executes the AFs Tier 4 \
decompression as part of the broader Comprehensive Airman Fitness. Specifically targeting \
redeployers resiliency impacts due to family separations and life challenges associated with \
deployments as well as the demands associated with combat operations. 2day program. 4 calen dar \
days which includes 2 days for trave l 72 hrs in Germany 10 hrs of program and 62 hrs of liberty \
Group discussion s not lectures led by a Mission Set Manager from the samesimilar mission set \
along with a Resiliency Technician no PowerPoints. Strength based focus to bridge positive \
combat and home front skillsets. No retribution or formal record of discussions that can be \
associated with members direct comments. The attendance is not recorded in the members medical record. \
Members can receive additional referrals to helping agencies as indicateddesired. One onone time is \
available but not mandated with the DTC mental health p ersonnel chaplain resiliency technicians and \
mission set managers. Experiential outing to a German city for a meal shopping r ecreation conversation etc. \
Civilian attire is the UOD throughout the course The DTC originally accepted members by line remark designation in \
official orders based on deployed duty or AFSC. Since inception AFCENT has been the primary theater of fo cus. \
Expansion of services has adjusted to include wider affected careerspersonnel and geographical locations i.e. \
AFAFRICA. Deployed commanders may nominate additional attendees based on exposure to incidents difficulty with \
stressors or concerns for home front transition'"

input_ids = tokenizer.encode(input_text, return_tensors="pt")

# Generate a response
output = model.generate(input_ids, max_length=500, max_new_tokens = 300, temperature=0.5, num_return_sequences=1)

# Decode the response
response = tokenizer.decode(output[0], skip_special_tokens=True)

dolly_7b_df = pd.DataFrame({'Generated Text': [response]})
dolly_7b_df

c:\Users\samuel.cannon\AppData\Local\anaconda3\envs\grok\Lib\site-packages\transformers\generation\configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Both `max_new_tokens` (=300) and `max_length`(=500) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


c:\Users\samuel.cannon\AppData\Local\anaconda3\envs\grok\Lib\site-packages\transformers\generation\utils.py:1460: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


,Generated Text
0,summarize the following text: 'TALKING PAPER ON THE DEPLOYMENT TRANSITION CENTER Since July 1 2010 the Air Force Deployment Transition Center DTC has provided world class reintegration and decompression to over 13848 redeployers. Over 800 Navy and USMC have also used the DTC as third location decompression The mission of the DTC is to provide critical reintegration skills and decompression opportunities for redeployers. Accomplished through four focused priorities Enhance People Build Community Flex and Adapt to Mission Needs and Conserve Resources. DTC curriculum executes the AFs Tier 4 decompression as part of the broader Comprehensive Airman Fitness. Specifically targeting redeployers resiliency impacts due to family separations and life challenges associated with deployments as well as the demands associated with combat operations. 2day program. 4 calen dar days which includes 2 days for trave l 72 hrs in Germany 10 hrs of program and 62 hrs of liberty Group discussion s not lectures led by a Mission Set Manager from the samesimilar mission set along with a Resiliency Technician no PowerPoints. Strength based focus to bridge positive combat and home front skillsets. No retribution or formal record of discussions that can be associated with members direct comments. The attendance is not recorded in the members medical record. Members can receive additional referrals to helping agencies as indicateddesired. One onone time is available but not mandated with the DTC mental health p ersonnel chaplain resiliency technicians and mission set managers. Experiential outing to a German city for a meal shopping r ecreation conversation etc. Civilian attire is the UOD throughout the course The DTC originally accepted members by line remark designation in official orders based on deployed duty or AFSC. Since inception AFCENT has been the primary theater of fo cus. Expansion of services has adjusted to include wider affected careerspersonnel and geographical locations i.e. AFAFRICA. Deployed commanders may nominate additional attendees based on exposure to incidents difficulty with stressors or concerns for home front transition''. The DTC is a critical component of the Air Force's comprehensive reintegration and decompression program for redeployers. The DTC provides world class reintegration and decompression to over 13848 redeployers. The mission of the DTC is to provide critical reintegration skills and decompression opportunities for redeployers. Accomplished through four focused priorities Enhance People Build Community Flex and Adapt to Mission Needs and Conserve Resources. DTC curriculum executes the AFs Tier 4 decompression as part of the broader Comprehensive Airman Fitness. Specifically targeting redeployers resiliency impacts due to family separations and life challenges associated with deployments as well as the demands associated with combat operations. 2day program. 4 calen dar days which includes 2 days for trave l 72 hrs in Germany 10 hrs of program and 62 hrs of liberty Group discussion s not lectures led by a Mission Set Manager from the samesimilar mission set along with a Resiliency Technician no PowerPoints. Strength based focus to bridge positive combat and home front skillsets. No retribution or formal record of discussions that can be associated with members direct comments. The attendance is not recorded in the members medical record. Members can receive additional referrals to helping agencies as indicateddesired. One onone time is available but not mandated with the DTC mental health p ersonnel chaplain resiliency technicians and mission set managers. Experiential outing to a German city


In [35]:
#dolly_7b_df['Response'] = dolly_7b_df['Generated Text'].str.replace('\n', ' ')
dolly_7b_df['Summary'] = dolly_7b_df['Generated Text'].str.extract(r'(The DTC is.*)')
dolly_7b_df

,Generated Text,Summary
0,summarize the following text: 'TALKING PAPER ON THE DEPLOYMENT TRANSITION CENTER Since July 1 2010 the Air Force Deployment Transition Center DTC has provided world class reintegration and decompression to over 13848 redeployers. Over 800 Navy and USMC have also used the DTC as third location decompression The mission of the DTC is to provide critical reintegration skills and decompression opportunities for redeployers. Accomplished through four focused priorities Enhance People Build Community Flex and Adapt to Mission Needs and Conserve Resources. DTC curriculum executes the AFs Tier 4 decompression as part of the broader Comprehensive Airman Fitness. Specifically targeting redeployers resiliency impacts due to family separations and life challenges associated with deployments as well as the demands associated with combat operations. 2day program. 4 calen dar days which includes 2 days for trave l 72 hrs in Germany 10 hrs of program and 62 hrs of liberty Group discussion s not lectures led by a Mission Set Manager from the samesimilar mission set along with a Resiliency Technician no PowerPoints. Strength based focus to bridge positive combat and home front skillsets. No retribution or formal record of discussions that can be associated with members direct comments. The attendance is not recorded in the members medical record. Members can receive additional referrals to helping agencies as indicateddesired. One onone time is available but not mandated with the DTC mental health p ersonnel chaplain resiliency technicians and mission set managers. Experiential outing to a German city for a meal shopping r ecreation conversation etc. Civilian attire is the UOD throughout the course The DTC originally accepted members by line remark designation in official orders based on deployed duty or AFSC. Since inception AFCENT has been the primary theater of fo cus. Expansion of services has adjusted to include wider affected careerspersonnel and geographical locations i.e. AFAFRICA. Deployed commanders may nominate additional attendees based on exposure to incidents difficulty with stressors or concerns for home front transition''. The DTC is a critical component of the Air Force's comprehensive reintegration and decompression program for redeployers. The DTC provides world class reintegration and decompression to over 13848 redeployers. The mission of the DTC is to provide critical reintegration skills and decompression opportunities for redeployers. Accomplished through four focused priorities Enhance People Build Community Flex and Adapt to Mission Needs and Conserve Resources. DTC curriculum executes the AFs Tier 4 decompression as part of the broader Comprehensive Airman Fitness. Specifically targeting redeployers resiliency impacts due to family separations and life challenges associated with deployments as well as the demands associated with combat operations. 2day program. 4 calen dar days which includes 2 days for trave l 72 hrs in Germany 10 hrs of program and 62 hrs of liberty Group discussion s not lectures led by a Mission Set Manager from the samesimilar mission set along with a Resiliency Technician no PowerPoints. Strength based focus to bridge positive combat and home front skillsets. No retribution or formal record of discussions that can be associated with members direct comments. The attendance is not recorded in the members medical record. Members can receive additional referrals to helping agencies as indicateddesired. One onone time is available but not mandated with the DTC mental health p ersonnel chaplain resiliency technicians and mission set managers. Experiential outing to a German city,The DTC is a critical component of the Air Force's comprehensive reintegration and decompression program for redeployers. The DTC provides world class reintegration and decompression to over 13848 redeployers. The mission of the DTC is to provide critical reintegration skills and decompression opportu

In [36]:
pwd

'c:\\Users\\samuel.cannon\\OneDrive - LinQuest Corporation\\Desktop\\decorations-generator\\decorations-generator'

In [40]:
# Replace 'filename.xlsx' with the path to the Excel file you want to import
summary_df = pd.read_excel(r'C:\Users\samuel.cannon\OneDrive - LinQuest Corporation\Desktop\decorations-generator\decorations-generator\model_summary_table.xlsx')

# Display the first few rows of the DataFrame
summary_df.head()

,model,number_of_parameters,time_to_inference,prompt,summary_example,temperature_range_tested
0,microsoft/Phi-3-mini-4k-instruct,4 billion,~60-70 minutes,"""summarize the following text:""","""The Air Force Deployment Transition Center (DTC), established on July 1, 2010, has provided over 13,848 redeployers with reintegration and decompression since its inception, attracting 800 Navy and USMC personnel as a third location decompression site. The DTC's mission is to deliver critical reintegration skills and decompression opportunities through four focused priorities: Enhance People, Build Community, Flex and Adapt to Mission Needs, and Conserve Resources. It executes the Air Force's Tier 4 decompression within Comprehensive Airman Fitness, targeting resilience for redeployers due to family separations and deployment life challenges, including combat-related demands. The 2-day program consists of 72 hours, noting a 2-day structure with a combination of program time and liberty time. Program activities involve group discussions led by a Mission Set Manager and Resiliency Technician, emphasizing a strength-based approach to assimilate combat and homefront skills, without recording individual discussions or formal attendance. A unique experiential outing in a German city is included for members to experience local culture. Uniformly, the DTC formerly used line remark designations in official orders and expanded in response to the expanding need for wider service beyond the Af-Europe (formerly AFCENT) theater. The DTC now supports a broader range of mission areas and geographical locations, including the AF Africa (AFRICA), allowing deploying commanders to nominate additional attendees as they assess individuals' exposure to incidents and their homefront transition concerns. The DTC's program and services aim to foster a supportive environment that facilitates a successful reintegration for service members and their families.""",.1 - .5
1,databricks-dolly-3b,3 billion,~3-5 minutes,"""summarize the following text contained in the ticks:""","""The DTC has a unique approach to reintegration and decompression that is designed to meet the specific needs of redeployers.1The curriculum is delivered in a non-traditional format of small group discussions and experiential learning.2The DTC is not affiliated with any religious organization.3\n\nThe conference will include plenary sessions, breakout sessions, and a networking reception on the evening of the 20th.""",.1 - .3
2,databricks-dolly-7b,7 billion,~20-30 minutes,"""summarize the following text:""","""The DTC is a critical component of the Air Force's comprehensive reintegration and decompression program for redeployers. The DTC provides world class reintegration and decompression to over 13848 redeployers. The mission of the DTC is to provide critical reintegration skills and decompression opportunities for redeployers. Accomplished through four focused priorities Enhance People Build Community Flex and Adapt to Mission Needs and Conserve Resources. DTC curriculum executes the AFs Tier 4 decompression as part of the broader Comprehensive Airman Fitness. Specifically targeting redeployers resiliency impacts due to family separations and life challenges associated with deployments as well as the demands associated with combat operations. 2day program. 4 calen dar days which includes 2 days for trave l 72 hrs in Germany 10 hrs of program and 62 hrs of liberty Group discussion s not lectures led by a Mission Set Manager from the samesimilar mission set along with a Resiliency Technician no PowerPoints. Strength based focus to bridge positive combat and home front skillsets. No retribution or formal record of discussions that can be associated with members direct comments. The attendance is not recorded in the members medical record. Members can receive additional referrals to helping agencies as indicateddesired. One onone time is available but not mandated with the DTC mental health p erso